<a href="https://colab.research.google.com/github/stox74/hoyoung_park/blob/master/Step_1_%EC%B0%B8%EA%B3%A0%EC%82%AC%EC%A0%84_%EA%B5%AC%EC%B6%95%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

!pip install konlpy

import pandas as pd
import numpy as np
import re

from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from collections import Counter

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.feature_extraction.text import TfidfVectorizer

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
     |████████████████████████████████| 19.4MB 217kB/s 
     |████████████████████████████████| 2.3MB 24.6MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [0]:
def remove_words(new_text):
    stop_words = ['스탁론', '탁론']
    for stopword in stop_words:
        if stopword in new_text:
            return False 
    return True
    
def check_len(str_1, len_num):
    if len(str_1) > len_num:
        return True 
    return False
  

def make_noun_doc(text):
    twitter = Okt()
    text_nouns = twitter.nouns(text)
    noun_count = Counter(text_nouns)
    str_result = ' '.join(text_nouns)
  
    return text_nouns, str_result, noun_count

def sentiment_analyst(df, len_num):
  
    df[0] = df['text']
    df[0] = df[0].drop_duplicates()
    df = df.dropna()
    df[0] = df[0].apply(lambda x: re.sub('[^가-힣0-9a-zA-Z\\s]', '', x))
    df[0] = df[0].apply(lambda x: ' '.join(x.split()))
  
    df['remove_word'] = df[0].apply(lambda x: remove_words(x))
    df = df[df['remove_word'] == True]
  
    #print(df)
    df['len_check'] = df[0].apply(lambda x: check_len(x, len_num))
    df = df[df['len_check'] == True]
    txt_list = df[0].values.tolist()
  
   # str, int , float, list, dict
    noun_list = []
    sentence = []
    noun_cnt = []
  
  #print(df)
  
    for i in range(len(txt_list)):
         
        text = txt_list[i]
        nouns, str_, noun_count = make_noun_doc(text)
        noun_list.append(nouns)
        sentence.append(str_)
        noun_cnt.append(noun_count)
    noun_count_df = (pd.DataFrame(noun_cnt)).T.fillna(0)
    noun_count_df['freq'] = noun_count_df.sum(axis=1)
    return sentence


def get_senti_score(text, dictionary):
    text_nouns, str_result, noun_count = make_noun_doc(text)
    print(text_nouns)
    sent_df = dictionary
    try:
        tmp_df = sent_df.loc[text_nouns] # text_nouns는 index리스트!
        tmp_df= tmp_df.dropna()
        print(tmp_df)
        result =  tmp_df[['pos', 'neg','neu_score','pos_score','neg_score']].sum().values.tolist()
        print(result)
    except:
        result = [0.0,0.0,0.0]
  
    return result

In [0]:
news_df = pd.read_excel('gdrive/My Drive/Research/세미나_1_TF_IDF/sample_data/news_sample.xlsx', index_col ='Date')
news_df=news_df.rename(columns={'기사내용': 'text'})

In [11]:
news_df

,text,label
Date,,
20100216,2003년 ‘카드 사태’가 터지면서 경제성장률이 외환위기 이후 최저인 2.8%까지 ...,1
20100321,[아시아경제 이초희 기자]출구전략의 핵심인 기준금리 인상이 하반기 이후로 점쳐지는 ...,1
20100331,[아시아경제 김민진 기자] 주요국 중앙은행들은 지난해 하반기 이후 금융위기 대응으로...,-1
20100507,세계 금융시장이 그리스 사태로 출렁이는 가운데 정부가 대책 마련에 고심하고 있다.정...,1
20100525,[아시아경제 김남현 기자] 금융통화위원회 의사록2010년도 제8차 회의1. 일 자 ...,0
...,...,...
20111214,외국계 투자은행들이 내년 한국은행의 기준금리 인하 가능성을 높게 점치는 가운데 스탠...,1
20111214,유로존 사태 해결 난망 내년에도 세계 경제 암울 투자는 최악상황까지 가정 욕심...,1
20111227,12월 8일 한국은행. 김중수 총재가 서울 중구 한은 건물 기자회견장에 들어섰다. ...,0


In [12]:
pos_df = news_df[news_df['label'] == 1] ## 금리인하
neg_df = news_df[news_df['label'] == -1] ## 금리상승

pos_sentence = sentiment_analyst(pos_df, 10)
neg_sentence = sentiment_analyst(neg_df, 10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
pos_sentence = ' '.join(pos_sentence)
neg_sentence = ' '.join(neg_sentence)

In [15]:
neg_sentence

'아시아 경제 김민 진 기자 국 중앙은행 지난해 하반기 이후 금융위기 대응 취했던 각종 조치 정상화 것 정상화 단계 및 속도 각국 사정 출구 전략 본격 측면 시사 바 한국은행 발표 통화 신용 정책 보고서 금융위기 진원 지인 미국 유로 지역 및 영국 등 금융시장 상황 개선 각종 유동성 조치 정상화 우선 추진 호주 노르웨이 이스라엘 등 상대 경제위기 여파 국가 예방 금리인하 정책 선회 정책 금리 인상 중국 인도 성장 국가 금융위기 이후 침체 우려 준 율과 대출 금리 인하 적극 금융 완화 정책 최근 효과 보고 은 준 매입 등 지원 위해 도입 각종 단기 금융시장 조치 지난 종료 영국 영란 은행 자산 매입 프로그램 통한 매입 지속 유로 지역 경우 위기 대응 차원 도입 및 개월 만가 거래 활용 장기 자금 공급 조작 지난해 종료 개월 만기 거래 대해 서도 고정 금리 입찰 전액 배정 방식 변동 금리 입찰 방식 정상 예정 또한 준 유로 지역 주택 모기지 증권 버드 본드 등 매입 조치 달 각각 종료 예정 영란 은행 경우 회사채 매입 조치 종료 시한 잔 매입 한도 회사채 시장 상황 개선 추가 매입 가능성 경제 안정국 분류 이스라엘 중앙은행 지난해 정책 금리 포인트 두 차례 인상 안착 호주 준비 은행 지난해 포인트 네 차례 금리 인상 노르웨이 중앙은행 시기 두 차례 인상 현재 호주 노르웨이 정책 금리 각각 다중 국 인민 은행 금융위기 초기 적극 대응 정책 대응 힘 중국 경제 지난해 목표 치 를 를 기록 통화 증가 율도 목표 치 내외 대폭 상회 를 기록 준비 은행 예상 회복 이 소비자 물가 상승 준율 인상 매입 금리 금리 각각 인상 유동성 관리 김민 진 기자 한국은행 금융통화위원회 본회의 기준금리 연 로 포인트 인상 결정 액 한도 대출 금리 현 수준 유지 은 기준금리 인상 것 지난 이후 개월 만이 당시 은 기준금리 로 리먼 브러더스 파산 금융위기 닥치 다음해 총 차례 금리인하 단행 기준금리 로 뒤 개월 연속 결기 이번 기준금리 인상 조치 국제 금융위기 극복 비상조치 대부분 종료 축소

In [16]:
corpus = [pos_sentence,neg_sentence]

tfidfv = TfidfVectorizer().fit(corpus)
# tfidf학습 후 array형태로 변환해준다
array_result = tfidfv.transform(corpus).toarray()
# tfidf에서 학습한 단어들
print(tfidfv.vocabulary_)
feature_names = tfidfv.get_feature_names()
# tfidf matrix를 dataframe형태로 이쁘게 가공해준다
tfidf_matrix = pd.DataFrame(array_result, columns = feature_names)
# 행렬을 보기 좋게 transpose해준다
tfidf_matrix = tfidf_matrix.T
# 컬럼 이름도 변경해준다
tfidf_matrix.columns = ['pos','neg']

# 각 긍정/부정에서 tfidf점수가 높은 상위 n개 단어 추출하기 위함
top_n = 100
# 각 긍정/부정에서 tfidf점수가 높은 상위 n개 단어 series를 저장
result_list = []
for i in tfidf_matrix.columns:
  # 각 긍정/부정에서 tfidf점수가 높은 상위 n개 단어 추출
  result_list.append(tfidf_matrix[i].sort_values(ascending=False)[:top_n])
  # 아래 코드는 tfidf가 일정수준 이상인 단어만 추출
  # tfidf_matrix.loc[tfidf_matrix[i] >= 0.2,i]
# 긍정/부정에서 tfidf점수가 높은 상위 단어결과 합치기
sent_df = pd.concat(result_list,axis=1)
# 긍정 또는 부정에서만 각각 쓰이는 단어의 경우 pos,neg에 값이 NaN이므로 이 부분을 0으로 채워준다
sent_df = sent_df.fillna(0)
# 감성중립스코어 컬럼 생성 및 초기화
sent_df['neu_score'] = np.nan
# 감성긍정스코어 컬럼 생성 및 초기화
sent_df['pos_score'] = np.nan
# 감성부정스코어 컬럼 생성 및 초기화
sent_df['neg_score'] = np.nan
# 긍정과 부정에 모두 tfidf점수가 있는 단어는 중립스코어 1로 설정
#sent_df.loc[(sent_df['pos'] != 0) & (sent_df['neg'] != 0),'neu_score'] = 1
# 긍정에만 tfidf점수가 있는 단어는 긍정스코어 1로 설정
#sent_df.loc[(sent_df['pos'] != 0) & (sent_df['neg'] == 0),'pos_score'] = 1
# 부정에만 tfidf점수가 있는 단어는 부정스코어 1로 설정
#sent_df.loc[(sent_df['pos'] == 0) & (sent_df['neg'] != 0),'neg_score'] = 1

sent_df['sent_ratio'] = sent_df['pos']/(sent_df['pos'] + sent_df['neg'])

sent_df.loc[sent_df['sent_ratio'] >= 0.5, 'pos_score' ] =1
sent_df.loc[sent_df['sent_ratio'] <= 0.5, 'neg_score' ] =1
#sent_df.loc[(sent_df['sent_ratio'] > 0.35) & (sent_df['sent_ratio'] < 0.65), 'neu_score' ] =1

sent_df = sent_df.fillna(0)

{'카드': 1877, '사태': 909, '경제성장률': 118, '외환위기': 1311, '이후': 1460, '최저': 1845, '곤두박질': 159, '한국은행': 2021, '기준금리': 319, '경기': 109, '서서히': 957, '시작': 1093, '당시': 440, '수준': 1043, '추가': 1846, '인하': 1477, '더블딥': 492, '우려': 1322, '일본': 1484, '장기': 1538, '불황': 867, '도래': 500, '비관': 873, '전망': 1582, '국내': 220, '금융시장': 274, '언론': 1204, '지배': 1747, '때문': 551, '회복': 2122, '부동산': 825, '시장': 1094, '개월': 68, '동안': 527, '금리': 267, '리지': 580, '차례': 1787, '인상': 1465, '단행': 429, '상승세': 925, '가격': 1, '잡기': 1535, '부족': 841, '강력': 51, '대출': 481, '규제': 245, '이번': 1420, '주식시장': 1693, '펀드': 1959, '광풍': 196, '글로벌': 265, '금융위기': 276, '사상': 895, '지난해': 1744, '전기': 1577, '대비': 463, '성장': 989, '기록': 299, '정도': 1614, '정부': 1618, '올해': 1297, '잠재': 1532, '안팎': 1172, '지난': 1741, '금융통화위원회': 279, '동결': 519, '경제위기': 120, '금리인하': 268, '지연': 1753, '모양새': 662, '닮은꼴': 433, '관건': 187, '후반전': 2134, '화로': 2098, '자산': 1516, '거품': 80, '후유증': 2135, '재현': 1560, '과잉': 183, '유동성': 1369, '반드시': 725, '대가': 445, '주장': 1698, '상황': 938, '크게

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## 금리의 안상, 인하관련 단어의 중요도가 scoring 된 사전이 만들어 진다.
## 사전이 경제적인 함의를 담고 있는지 개별적으로 확인해 본다.


In [17]:
# 금리의 안상, 인하관련 단어의 중요도가 scoring 된 사전이 만들어 진다  
sent_df

,pos,neg,neu_score,pos_score,neg_score,sent_ratio
가격,0.037066,0.000000,0.0,1.0,0.0,1.000000
가계,0.000000,0.078030,0.0,0.0,1.0,0.000000
가계부채,0.000000,0.041616,0.0,0.0,1.0,0.000000
가능성,0.126023,0.046818,0.0,1.0,0.0,0.729126
각종,0.000000,0.031212,0.0,0.0,1.0,0.000000
...,...,...,...,...,...,...
확대,0.034100,0.000000,0.0,1.0,0.0,1.000000
환율,0.039592,0.000000,0.0,1.0,0.0,1.000000
활용,0.000000,0.029245,0.0,0.0,1.0,0.000000
회복,0.000000,0.052020,0.0,0.0,1.0,0.000000
